In [1]:
# Run the cell to install dependencies and import packages 
import requests
import lxml.html as lh
import pandas as pd
!rm -r data # in case you are running this notebook in environment with folder

rm: cannot remove 'data': No such file or directory


This dataset provides a list of nations (ranked in order by medal wins) who have competed in the Olympics. The columns are Rank, Nation, Number of Games, Gold Medal Count, Silver Medal Count, Bronze Medal Count, and Total Medal Count. There are currently no values missing from the dataset.

In [17]:
# Scrape data from website 

# Looking at 22W medians from ORC page
url='https://www.topendsports.com/events/summer/medal-tally/all-time-all.htm'

# Create "page" to handle the contents of website
page = requests.get(url)

# Use "document" to store contents of website
document = lh.fromstring(page.content)

# Parse data stored between </tr>'s of HTML (to scrape from HTML table)
table_elements = document.xpath('//tr')

# Perform sanity check to make sure number of columns in first 5 rows of table are the same (should be 4)
print([len(row) for row in table_elements[:6]])

# Perform sanity check on what we've scraped
print(table_elements[0].text_content())
print(table_elements[1].text_content())

[7, 7, 7, 7, 7, 7]

Rank
Nation (NOC)
No. of Games
Gold
Silver
Bronze
Total


1
United States 
28
1061
836
739
2636



In [18]:
# Write data to a dataframe 

# Create a list called table that will hold all our data
table = []

# ColNum will keep track of which column in the table we are looking at
colNum = 0

# For each column, store the first element (titles of the columns) and an empty list in "table"
for firstRow in table_elements[0]:
    colNum += 1
    name = firstRow.text_content()
    print('%d:"%s"' % (colNum, name))  # print column index and title of column for a sanity check
    table.append((name, []))

# Since first row is the header, data is stored on the second row onwards
for rowNum in range(1, len(table_elements)):

    thisRow = table_elements[rowNum]  # thisRow stores all columns in current row
    
    # Ensures that rows have 4 columns (writing defensible code)
    if len(thisRow) != 7:
        break

    colNum = 0  # colNum keeps track of column as we traverse this row

    # Iterate through each element of current row
    for elem in thisRow:
        data = elem.text_content()
        try: # processing enrollment data 
            data = int(data)
        except:
            pass
        table[colNum][1].append(data)  # Append the data to the list of this current column
        colNum += 1  # Increment i for the next column

print(table)

1:"Rank"
2:"Nation (NOC)"
3:"No. of Games"
4:"Gold"
5:"Silver"
6:"Bronze"
7:"Total"
[('Rank', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]), ('Nation (NOC)', ['United States ', 'Soviet Union ', 'Germany ', 'Great Britain ', 'China ', 'France ', 'Italy ', 'Hungary ', 'Russia ', 'Japan ', 'Australia ', 'East Germany ', 'Sweden ', 'Finland ', 'South Korea ', 'Netherlands ', 'Romania ', 'Cuba ', 'Poland ', 'Canad

In [19]:
# Convert table to pandas data frame
Dict = {title: column for (title, column) in table}
df = pd.DataFrame(Dict)

df.head()


,Rank,Nation (NOC),No. of Games,Gold,Silver,Bronze,Total
0,1,United States,28,1061,836,739,2636
1,2,Soviet Union,10,440,357,325,1122
2,3,Germany,25,285,326,363,974
3,4,Great Britain,29,285,315,315,915
4,5,China,11,262,199,173,634


In [20]:
# write output to file
!mkdir data
df.to_csv("data/rawData.csv", index=False)

mkdir: cannot create directory ‘data’: File exists


In [28]:
# Load in file and rename columns 
dataTable = pd.read_csv("data/rawData.csv") # read in raw data file
dataTable.head()

,Rank,Nation (NOC),No. of Games,Gold,Silver,Bronze,Total
0,1,United States,28,1061,836,739,2636
1,2,Soviet Union,10,440,357,325,1122
2,3,Germany,25,285,326,363,974
3,4,Great Britain,29,285,315,315,915
4,5,China,11,262,199,173,634


In [29]:
# [1] Processing data by separating Wins (Gold) or Losses (Silver/Bronze) --> following Talladega Night's "second is the first loser" idea
# [1] Processing data by separating course info (need domain knowledge on the data)
losses = []

for i, row in dataTable.iterrows(): 
    if i == 0: 
      print(row)
    silver = row['Silver']
    bronze = row['Bronze']
    losses.append(silver+bronze)

print(losses)
dataTable.rename(columns={'gold':'Wins'}, inplace=True) # rename columns
dataTable.insert(5, "Losses", losses) # insert column
dataTable.drop(["Silver"], axis=1, inplace=True) # drop repetitive info
dataTable.drop(["Bronze"], axis=1, inplace=True) # drop repetitive info
print("resulting df: ")
print(dataTable.head())

Rank                         1
Nation (NOC)    United States 
No. of Games                28
Gold                      1061
Silver                     836
Bronze                     739
Total                     2636
Name: 0, dtype: object
[1575, 682, 689, 630, 372, 538, 401, 330, 335, 328, 390, 256, 355, 204, 191, 226, 218, 156, 226, 255, 100, 170, 152, 86, 94, 157, 119, 112, 63, 113, 104, 86, 78, 62, 62, 61, 52, 35, 56, 75, 48, 38, 57, 27, 72, 60, 24, 30, 26, 26, 25, 23, 22, 30, 29, 20, 8, 8, 42, 29, 17, 20, 18, 28, 23, 12, 10, 17, 7, 24, 16, 16, 10, 9, 7, 5, 3, 2, 0, 28, 16, 11, 8, 8, 7, 6, 1, 1, 13, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 13, 7, 6, 5, 5, 4, 4, 4, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
resulting df: 
   Rank    Nation (NOC)  No. of Games  Gold  Losses  Total
0     1  United States             28  1061    1575   2636
1     2   Soviet Union             10   440     682   1122
2     3        Germ

In [38]:
# [2] processing : adding percent of total gold medals category
percentOfGold = []

for i, row in dataTable.iterrows():
    gold = row["Gold"]
    total = row["Total"]
    if (total==0 or gold==0):
      percent = 0
    else: percent = gold/total * 100
    percentOfGold.append(percent)
dataTable.insert(6, "Percent of Gold Medals", percentOfGold)
dataTable.head()

,Rank,Nation (NOC),No. of Games,Gold,Losses,Total,Percent of Gold Medals
0,1,United States,28,1061,1575,2636,40.250379
1,2,Soviet Union,10,440,682,1122,39.215686
2,3,Germany,25,285,689,974,29.260780
3,4,Great Britain,29,285,630,915,31.147541
4,5,China,11,262,372,634,41.324921


In [59]:
# [3] augmentation: add regions
continents = []
region_dict = {
    'United States': 'Americas', 'Soviet Union': 'Europe', 'Germany': 'Europe', 'Great Britain': 'Europe', 'China':'Asia', 'France':'Europe', 'Italy':'Europe', 
    'Hungary':'Europe', 'Russia':'Europe','Japan': 'Asia', 'Australia':'Oceania', 'East Germany':'Europe', 'Sweden': 'Europe', 'Finland': 'Europe',	'South Korea': 'Asia',
    'Netherlands': 'Europe', 'Romania':'Europe', 'Cuba':'Americas', 'Poland':'Europe', 'Canada':'Americas', 'Norway':'Europe', 'Bulgaria':'Europe', 'Switzerland':'Europe',
    'New Zealand':'Oceania', 'Czechoslovakia':'Europe', 'Denmark':'Europe', 'Spain':'Europe', 'Belgium':'Europe', 'Türkiye':'Europe', 'Brazil':'Americas', 'Ukraine':'Europe',
    'Greece':'Europe', 'Kenya':'Africa', 'Yugoslavia':'Europe', 'South Africa':'Africa', 'Jamaica':'Americas', 'Iran':'Asia', 'Ethiopia':'Africa', 'Argentina':'Americas',
    'Austria':'Europe', 'Czechia':'Europe', 'North Korea':'Asia', 'Kazakhstan':'Europe', 'Croatia':'Europe', 'Belarus':'Europe', 'Mexico':'Americas', 'Ireland':'Europe',
    'Georgia':'Europe', 'Uzbekistan':'Europe', 'Estonia':'Europe', 'Thailand':'Asia', 'India':'Asia', 'Slovakia':'Europe', 'Egypt':'Africa', 'Indonesia':'Asia', 'Slovenia':'Europe',
    'Bahamas':'Americas', 'Azerbaijan':'Europe', 'Chinese Taipei':'Asia', 'Mixed team':'Mixed', 'Morocco':'Africa', 'Lithuania':'Europe', 'Serbia':'Europe', 'Colombia':'Americas',
    'Portugal':'Europe', 'Algeria':'Africa', 'Tunisia':'Africa', 'Latvia':'Europe', 'Uganda':'Africa', 'Nigeria':'Africa', 'Venezuela':'Americas', 'Trinidad and Tobago':'Americas',
    'Israel':'Asia', 'Dominican Republic':'Americas', 'Pakistan':'Asia', 'Zimbabwe':'Africa', 'Cameroon':'Africa', 'Ecuador':'Americas'}
          
for i, row in dataTable.iterrows():
    nation = str(row["Nation (NOC)"])
    nation = nation.strip()
    if nation in region_dict: 
      region = region_dict[nation]
    else: region = 'N/A'
    continents.append(region)

dataTable.insert(2, "Continents", continents)
dataTable.head()

,Rank,Nation (NOC),Continents,No. of Games,Gold,Losses,Total,Percent of Gold Medals
0,1,United States,Americas,28,1061,1575,2636,40.250379
1,2,Soviet Union,Europe,10,440,682,1122,39.215686
2,3,Germany,Europe,25,285,689,974,29.260780
3,4,Great Britain,Europe,29,285,630,915,31.147541
4,5,China,Asia,11,262,372,634,41.324921


In [61]:
dataTable.rename(columns={'Percent of Gold Medals':'% of Medals that are Gold'}, inplace=True) # rename columns
print("resulting df: ")
print(dataTable.head())

resulting df: 
   Rank    Nation (NOC) Continents  No. of Games  Gold  Losses  Total  \
0     1  United States    Americas            28  1061    1575   2636   
1     2   Soviet Union      Europe            10   440     682   1122   
2     3        Germany      Europe            25   285     689    974   
3     4  Great Britain      Europe            29   285     630    915   
4     5          China        Asia            11   262     372    634   

   % of Medals that are Gold  
0                  40.250379  
1                  39.215686  
2                  29.260780  
3                  31.147541  
4                  41.324921  


In [63]:
# [4] get percent of total gold medals
percentTotalGold = []
sum = 0
for i, row in dataTable.iterrows():
    gold = int(row["Gold"])
    sum += gold

for i, row in dataTable.iterrows():
    gold = int(row["Gold"])
    percentTotalGold.append(gold/sum*100)

dataTable.insert(8, "% of All Gold Medals", percentTotalGold)
dataTable.head()

,Rank,Nation (NOC),Continents,No. of Games,Gold,Losses,Total,% of Medals that are Gold,% of All Gold Medals
0,1,United States,Americas,28,1061,1575,2636,40.250379,19.428676
1,2,Soviet Union,Europe,10,440,682,1122,39.215686,8.057132
2,3,Germany,Europe,25,285,689,974,29.260780,5.218824
3,4,Great Britain,Europe,29,285,630,915,31.147541,5.218824
4,5,China,Asia,11,262,372,634,41.324921,4.797656


In [66]:
# [5] Augmentation: is the country a real player? Add dictionary for countries in more/less than 5, 10, 15, 20, 25 games
numGamesDict = {5: 'Competed Litte', 10: 'Moderately Competed', 15: 'Average', 20: 'Competes Frequently', 25: 'Almost Always Competes'}
competitions = []

for i, row in dataTable.iterrows():
  games = int(row["No. of Games"])
  if games <= 5:
    key = 5
  elif games <= 10:
    key = 10
  elif games <= 15:
    key = 15
  elif games <= 20:
    key = 20
  else: key = 25
  competitiveness = numGamesDict[key]
  competitions.append(competitiveness)

dataTable.insert(4, "Frequency of Competition", competitions)
dataTable.head()

,Rank,Nation (NOC),Continents,No. of Games,Frequency of Competition,Gold,Losses,Total,% of Medals that are Gold,% of All Gold Medals
0,1,United States,Americas,28,Almost Always Competes,1061,1575,2636,40.250379,19.428676
1,2,Soviet Union,Europe,10,Moderately Competed,440,682,1122,39.215686,8.057132
2,3,Germany,Europe,25,Almost Always Competes,285,689,974,29.260780,5.218824
3,4,Great Britain,Europe,29,Almost Always Competes,285,630,915,31.147541,5.218824
4,5,China,Asia,11,Average,262,372,634,41.324921,4.797656


In [68]:
dataTable.rename(columns={'Gold':'Wins'}, inplace=True) # rename columns
dataTable.head()

,Rank,Nation (NOC),Continents,No. of Games,Frequency of Competition,Wins,Losses,Total,% of Medals that are Gold,% of All Gold Medals
0,1,United States,Americas,28,Almost Always Competes,1061,1575,2636,40.250379,19.428676
1,2,Soviet Union,Europe,10,Moderately Competed,440,682,1122,39.215686,8.057132
2,3,Germany,Europe,25,Almost Always Competes,285,689,974,29.260780,5.218824
3,4,Great Britain,Europe,29,Almost Always Competes,285,630,915,31.147541,5.218824
4,5,China,Asia,11,Average,262,372,634,41.324921,4.797656


I have decided to process my data by adding columns of Continents, Frequency of Competition, Wins (renamed from Gold), Losses, Percent of Medals earned that are gold, and percent of gold medals recieved out of all gold medals won. I think that this data will be helpful because we can now see the spread of wins/gold medals across continents rather than countries, perhaps seeing if there is a common winning trend in certain geographical regions/continents. Then, one could discover what factors (environmental, economic, governmental, or medical) that may contribute to this increased gold medal count. I simplified the Bronze and Silver medal categories to "Losses" so that I could just work with my Wins (Gold Medals) while still acknowledging the prescence of other medals recieved.   

In [72]:
# write output to file
!mkdir data
dataTable.to_csv("data/processedData1.csv", index=False)

mkdir: cannot create directory ‘data’: File exists
